In [1]:
import scipy
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

In [2]:
n=10000
x=np.arange(n)
y = np.random.normal(size=(1000,512))+1

In [3]:
y.shape

(1000, 512)

In [4]:
kernel = stats.gaussian_kde(y)

NameError: name 'stats' is not defined

In [5]:
sns.histplot(y)

<AxesSubplot:ylabel='Count'>

/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/IPython/core/pylabtools.py:151: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  fig.canvas.print_figure(bytes_io, **kw)


Error in callback <function flush_figures at 0x7f68a5c4f700> (for post_execute):


KeyboardInterrupt: 

In [ ]:
ymin=min(y)
ymax=max(y)
ymin,ymax

In [ ]:
points=np.linspace(ymin,ymax,2000)

In [8]:
from scipy import stats
from sklearn.neighbors import KernelDensity
#from sklearn.neighbors.kde import KernelDensity

In [ ]:
x2=np.random.normal(size=(2,512))

In [ ]:
y2=kernel(x2)

In [ ]:
plt.plot(points,y2)

In [16]:
def reparameterize(mu: Tensor, logvar: Tensor) -> Tensor:
    """
    Reparameterization trick to sample from N(mu, var) from
    N(0,1).
    :param mu: (Tensor) Mean of the latent Gaussian [B x D]
    :param logvar: (Tensor) Standard deviation of the latent Gaussian [B x D]
    :return: (Tensor) [B x D]
    """
    std = torch.exp(0.5 * logvar)
    eps = torch.randn_like(std)
    return eps * std + mu

In [17]:
def multivariate_gaussian(X, mu, var):
    """
    Computes the probability 
    density function of the examples X under the multivariate gaussian 
    distribution with parameters mu and var. If var is a matrix, it is
    treated as the covariance matrix. If var is a vector, it is treated
    as the var values of the variances in each dimension (a diagonal
    covariance matrix
    """
    
    k = len(mu)
    if var.ndim == 1:
        var = np.diag(var)
    
    X = X - mu
    p = (2* np.pi)**(-k/2) * np.linalg.det(var)**(-0.5) * \
        np.exp(-0.5 * np.sum(np.matmul(X, np.linalg.pinv(var)) * X, axis=1))
    
    return p

In [10]:
mu=np.zeros(512)
var=np.ones(512)
x=np.random.randn(1024,512)
x.shape,mu.shape,var.shape

((1024, 512), (512,), (512,))

In [29]:
multivariate_gaussian(x, mu, var)

array([1.40844311e-316, 2.16439932e-305, 3.51352169e-315, ...,
       0.00000000e+000, 1.67982320e-322, 3.19361341e-316])

In [117]:
from scipy.stats import multivariate_normal


In [118]:
def univariate_normal(x, mean, variance):
    """pdf of the univariate normal distribution."""
    return ((1. / np.sqrt(2 * np.pi * variance)) * 
            np.exp(-(x - mean)**2 / (2 * variance)))

In [33]:
mn=multivariate_normal(mean=mu, cov=var)

In [34]:
y = mn.pdf(x)

In [35]:
y

array([1.40844311e-316, 2.16439932e-305, 3.51352169e-315, ...,
       0.00000000e+000, 1.67982320e-322, 3.19361341e-316])

In [36]:
## checking probs

In [12]:
import sys
import numpy as np
sys.path.append("../")
from data.dataloader import *
from models.autoencoder_mu_var import *

In [13]:
weights="../ckpts/anamoly_road_512/lightning_logs/version_1/checkpoints/epoch=209-step=202020.ckpt"
model = Autoencoder.load_from_checkpoint(weights)

Downloading: "https://download.pytorch.org/models/vgg19_bn-c79401a0.pth" to /home/ec2-user/.cache/torch/hub/checkpoints/vgg19_bn-c79401a0.pth


In [14]:
def get_features(model,dloader,keepk=None):
    ## Train features for checking 
    features_mu=[]
    features_std=[]
    features=[]
    labels=[]
    model.cuda()
    model.eval()
    for i,item in tqdm(enumerate(tqdm(dloader))):
        if i>10:
            break
        x,label=item[0],item[1]
        model.cuda()
        model.eval()
        x=x.cuda()
        mu,logvar=model.encoder(x)
        z=reparameterize(mu,logvar).detach().cpu().numpy()
        std = torch.exp(0.5 * logvar)
        features_mu.append(mu.flatten(start_dim=1).detach().cpu().numpy())
        features_std.append(std.flatten(start_dim=1).detach().cpu().numpy())
        labels.append(label)
        features.append(z)
    
    features_mu=np.concatenate(features_mu,axis=0)
    features_std=np.concatenate(features_std,axis=0)
    features=np.concatenate(features,axis=0)
    labels=np.concatenate(labels,axis=0)

    mean=np.average(features_mu , axis=0)
    std=np.average(features_std , axis=0)
    
    return mean,std,features,labels

In [18]:
from tqdm import tqdm
train_loader,val_loader,_=get_data(32)
mean,var,features,labels=get_features(model,val_loader)
kernel=(mean,var)

11it [00:12,  1.13s/it] [00:12<02:40,  1.30it/s]
  5%|▌         | 11/219 [00:17<05:31,  1.59s/it]


In [19]:
mean.shape,var.shape,mean.min(),mean.max(),var.min(),var.max()

((512,), (512,), -0.5911034, 0.5383877, 0.028959421, 1.0022132)

In [20]:
features.shape

(352, 512)

In [24]:
kde = KernelDensity(kernel='gaussian', bandwidth=0.000001).fit(features)
score =  kde.score_samples(features)

In [26]:
score.shape
score

array([6597.1812455, 6597.1812455, 6597.1812455, 6597.1812455,
       6597.1812455, 6597.1812455, 6597.1812455, 6597.1812455,
       6597.1812455, 6597.1812455, 6597.1812455, 6597.1812455,
       6597.1812455, 6597.1812455, 6597.1812455, 6597.1812455,
       6597.1812455, 6597.1812455, 6597.1812455, 6597.1812455,
       6597.1812455, 6597.1812455, 6597.1812455, 6597.1812455,
       6597.1812455, 6597.1812455, 6597.1812455, 6597.1812455,
       6597.1812455, 6597.1812455, 6597.1812455, 6597.1812455,
       6597.1812455, 6597.1812455, 6597.1812455, 6597.1812455,
       6597.1812455, 6597.1812455, 6597.1812455, 6597.1812455,
       6597.1812455, 6597.1812455, 6597.1812455, 6597.1812455,
       6597.1812455, 6597.1812455, 6597.1812455, 6597.1812455,
       6597.1812455, 6597.1812455, 6597.1812455, 6597.1812455,
       6597.1812455, 6597.1812455, 6597.1812455, 6597.1812455,
       6597.1812455, 6597.1812455, 6597.1812455, 6597.1812455,
       6597.1812455, 6597.1812455, 6597.1812455, 6597.1

In [ ]:
  s = len(test_labels)
c = np.sum(test_labels==1)
g = c/s

thresh = np.percentile(test_score, int(g*100))
pred = (test_score < thresh).astype(int)
gt = test_labels.astype(int)

In [121]:
#mn=multivariate_normal(mean=mean, cov=var)

In [132]:
probs=[]
for index in range(features.shape[1]):
    prob=univariate_normal(features[:,index],mean=0,variance=1)
    #mn=multivariate_normal(mean=mean[index], cov=var[index])
    #prob = mn.pdf(features[:,index])
    probs.append(prob)
probs=np.stack(probs,axis=1)
probs.shape

(7008, 512)

In [140]:
probs.max(axis=1).max()

0.3989423

In [141]:
pdf=np.sum(probs,axis=1)/probs.shape[1]
pdf.shape

(7008,)

In [142]:
pdf.min(),pdf.max()

(0.2618318, 0.29970235)

In [143]:
np.max(probs,axis=1).min()

0.3988435

In [144]:
prob

array([0.07208881, 0.0397646 , 0.2803546 , ..., 0.3338421 , 0.31041545,
       0.37580782], dtype=float32)

In [45]:
prob.shape,prob.min(),prob.max()

((352,), 0.0, 9.635174465364923e-295)